# ML Pipeline Preparation
This is a simple machine learning script that looks at tweets during natural disasters and attempts to map them to 36 different categories. The result of this can then be used to help emergency services give aid to those who truly require it.

### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [87]:
# import libraries
import pandas as pd
import sqlalchemy
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [69]:
for col in Y.columns:
    print(col)
    print(Y[col].value_counts())
    print(' ')

request
0    21742
1     4474
Name: request, dtype: int64
 
offer
0    26098
1      118
Name: offer, dtype: int64
 
aid_related
0    15356
1    10860
Name: aid_related, dtype: int64
 
medical_help
0    24132
1     2084
Name: medical_help, dtype: int64
 
medical_products
0    24903
1     1313
Name: medical_products, dtype: int64
 
search_and_rescue
0    25492
1      724
Name: search_and_rescue, dtype: int64
 
security
0    25745
1      471
Name: security, dtype: int64
 
military
0    25356
1      860
Name: military, dtype: int64
 
water
0    24544
1     1672
Name: water, dtype: int64
 
food
0    23293
1     2923
Name: food, dtype: int64
 
shelter
0    23902
1     2314
Name: shelter, dtype: int64
 
clothing
0    25811
1      405
Name: clothing, dtype: int64
 
money
0    25612
1      604
Name: money, dtype: int64
 
missing_people
0    25918
1      298
Name: missing_people, dtype: int64
 
refugees
0    25341
1      875
Name: refugees, dtype: int64
 
death
0    25022
1     1194
Name: death,

In [77]:
# load data from database
engine = sqlalchemy.create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Message', engine)
X = df['message']
Y = df.iloc[:,6:] #Skip genre & related as these need to be one hot encoded then add later
Y_dummies = pd.get_dummies(df[['genre','related']].astype(str))
Y = Y.merge(Y_dummies, how='inner',left_index=True,right_index=True, validate='1:1')

print(Y.head())
print(X.head())

   request  offer  aid_related  medical_help  medical_products  \
0        0      0            0             0                 0   
1        0      0            1             0                 0   
2        0      0            0             0                 0   
3        1      0            1             0                 1   
4        0      0            0             0                 0   

   search_and_rescue  security  military  child_alone  water  ...  earthquake  \
0                  0         0         0            0      0  ...           0   
1                  0         0         0            0      0  ...           0   
2                  0         0         0            0      0  ...           0   
3                  0         0         0            0      0  ...           0   
4                  0         0         0            0      0  ...           0   

   cold  other_weather  direct_report  genre_direct  genre_news  genre_social  \
0     0              0             

### 2. A simple tokenization function to process text data

In [78]:
def tokenize(text):
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. A machine learning pipeline
This machine pipeline takes in the `message` column as input and output classification results on the other 36 categories in the dataset.

In [82]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=226)
pipeline.fit(X_train, y_train)
pipeline.predict(X[-2:]) #

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]],
      dtype=int64)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [84]:
y_pred = pipeline.predict(X_test)

In [85]:
for i in range(y_pred.shape[1]):
    col_name = y_test.columns[i]
    y_test_i = y_test.iloc[:,i]
    y_pred_i = [item[i] for item in y_pred]
    print(col_name + ":")
    print(classification_report(y_test_i, y_pred_i))

request:
              precision    recall  f1-score   support

           0       0.84      0.99      0.91      5414
           1       0.72      0.08      0.14      1140

    accuracy                           0.83      6554
   macro avg       0.78      0.53      0.52      6554
weighted avg       0.82      0.83      0.77      6554

offer:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1       0.00      0.00      0.00        30

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

aid_related:
              precision    recall  f1-score   support

           0       0.58      0.99      0.73      3767
           1       0.73      0.04      0.07      2787

    accuracy                           0.58      6554
   macro avg       0.65      0.51      0.40      6554
weighted avg       0.64      0.58      0.45  

c:\users\mnasra\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.99      6384
           1       0.00      0.00      0.00       170

    accuracy                           0.97      6554
   macro avg       0.49      0.50      0.49      6554
weighted avg       0.95      0.97      0.96      6554

missing_people:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6471
           1       0.00      0.00      0.00        83

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.97      0.99      0.98      6554

refugees:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6306
           1       0.00      0.00      0.00       248

    accuracy                           0.96      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.93      0.96      0.94     

              precision    recall  f1-score   support

           0       0.79      0.99      0.88      5065
           1       0.76      0.12      0.21      1489

    accuracy                           0.79      6554
   macro avg       0.78      0.56      0.55      6554
weighted avg       0.79      0.79      0.73      6554

related_1:
              precision    recall  f1-score   support

           0       0.40      0.66      0.50      1546
           1       0.87      0.69      0.77      5008

    accuracy                           0.69      6554
   macro avg       0.63      0.68      0.64      6554
weighted avg       0.76      0.69      0.71      6554

related_2:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6497
           1       0.39      0.30      0.34        57

    accuracy                           0.99      6554
   macro avg       0.69      0.65      0.67      6554
weighted avg       0.99      0.99      0.99      655

### 6. Improve your model
Use grid search to find better parameters. 

In [94]:
parameters = {
    
    'vect__ngram_range':[(1,1),(1,2),(2,2)],
    'tfidf__use_idf': (True, False), 
    'clf__estimator__p':[1,2]
}

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [96]:
cv.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
y_pred = cv.predict(X_test)
for i in range(y_pred.shape[1]):
    col_name = y_test.columns[i]
    y_test_i = y_test.iloc[:,i]
    y_pred_i = [item[i] for item in y_pred]
    print(col_name + ":")
    print(classification_report(y_test_i, y_pred_i))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [108]:
classification_labels = pipeline.predict(['help, i need to drink, the army took it all'])

In [109]:
pd.DataFrame(classification_labels.transpose()).merge(pd.DataFrame(Y.columns.transpose()),left_index=True,right_index=True)

,0_x,0_y
0,0,request
1,0,offer
2,0,aid_related
3,0,medical_help
4,0,medical_products
5,0,search_and_rescue
6,0,security
7,0,military
8,0,water
9,0,food


Index(['request', 'offer', 'aid_related', 'medical_help', 'medical_products',
       'search_and_rescue', 'security', 'military', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report', 'genre_direct', 'genre_news',
       'genre_social', 'related_0', 'related_1', 'related_2'],
      dtype='object')

### 9. Export your model as a pickle file

In [62]:
import pickle
with open('models/modelv1.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.